Multitask learning on the IMDB dataset using TensorFlow where we aim to predict the genre of the movie based on its review. 

In [ ]:
import tensorflow as tf
from tensorflow.keras.datasets import imdb
from tensorflow.keras.layers import Input, Embedding, Dense, GlobalMaxPooling1D, concatenate
from tensorflow.keras.models import Model

Load the IMDB dataset

In [ ]:
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=10000)

Pad the sequences to a maximum length of 100

In [ ]:

x_train = tf.keras.preprocessing.sequence.pad_sequences(x_train, maxlen=100)
x_test = tf.keras.preprocessing.sequence.pad_sequences(x_test, maxlen=100)

Define the input layer

In [ ]:
input_layer = Input(shape=(100,))

Define the shared embedding layer

In [ ]:

embedding_layer = Embedding(input_dim=10000, output_dim=32)(input_layer)

Define the first output head for sentiment analysis

In [ ]:

sentiment_output = Dense(1, activation='sigmoid', name='sentiment_output')(GlobalMaxPooling1D()(embedding_layer))

Define the second output head for movie genre classification

We assign each review to a movie genre based on its label index modulo 5, since there are 5 possible genre labels in the IMDB dataset.

In [ ]:

genre_output = Dense(5, activation='softmax', name='genre_output')(GlobalMaxPooling1D()(embedding_layer))

Define the model with two output heads

In [ ]:

model = Model(inputs=input_layer, outputs=[sentiment_output, genre_output])

Compile the model with two loss functions and metrics

In [ ]:

model.compile(optimizer='adam', loss={'sentiment_output': 'binary_crossentropy', 'genre_output': 'categorical_crossentropy'}, metrics={'sentiment_output': 'accuracy', 'genre_output': 'accuracy'})

Train the model with both tasks

In [ ]:
history = model.fit(x_train, {'sentiment_output': y_train, 'genre_output': y_train % 5}, validation_data=(x_test, {'sentiment_output': y_test, 'genre_output': y_test % 5}), epochs=10)